In [ ]:
!pip install --quiet -U flash-attn datasets accelerate peft bitsandbytes transformers langchain_community --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 32.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
ERROR:

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

from google.colab import drive
from google.colab import userdata
from huggingface_hub import login

In [ ]:
token = userdata.get('hf')
login(token=token)

In [ ]:
drive.mount('/content/drive')

In [ ]:
def get_model_and_tokenizer(model_id):

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    return model, tokenizer

In [ ]:
steps=400
base_model_id = ''
adapter_model_id = '' 
hub_model_id= f''
subfolder= f'checkpoint-{steps}'

In [ ]:
base_model, tokenizer = get_model_and_tokenizer(base_model_id)

In [ ]:
def save_peft_model(adapter_model_id, base_model, hub_model_id, subfolder='/'):
  tokenizer = AutoTokenizer.from_pretrained(adapter_model_id, subfolder=subfolder)
  base_model.resize_token_embeddings(max(len(tokenizer), base_model.config.vocab_size))
  model = PeftModel.from_pretrained(base_model, adapter_model_id, subfolder=subfolder)
  model = model.merge_and_unload()

  hf_token = token
  tokenizer.push_to_hub(hub_model_id, token=hf_token)
  model.push_to_hub(hub_model_id, token=hf_token)

  return model, tokenizer

In [ ]:
model, tokenizer = save_peft_model(adapter_model_id, base_model, hub_model_id, subfolder)